# Web Scraping de las fechas de publicación del inventario semanal de Investing.com

Este script utiliza `Selenium` para hacer un scraping dinámico de la página de **Investing.com**, extrayendo las fechas de publicación del inventario semanal. El script también usa un bucle para hacer scroll y cargar más datos a medida que avanza, hasta alcanzar la fecha deseada.

In [21]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.options import Options
from bs4 import BeautifulSoup

# Ruta a tu GeckoDriver
driver_path = 'C:/Users/Usuario/Desktop/Master Data& IA/APIS y Web scraping/chromedriver firefoxdriver/geckodriver.exe'
service = Service(driver_path)

# Ruta del perfil de Firefox
profile_path = 'C:/Users/Usuario/AppData/Roaming/Mozilla/Firefox/Profiles/prnq9tzg.default-release'

# Crear opciones de Firefox
firefox_options = Options()

# Ruta al binario de Firefox (el ejecutable)
firefox_binary_path = 'C:/Program Files/Mozilla Firefox/firefox.exe'
firefox_options.binary_location = firefox_binary_path

# Asignar el perfil de Firefox
firefox_options.profile = profile_path

# Iniciar Firefox con el perfil y las opciones configuradas
driver = webdriver.Firefox(service=service, options=firefox_options)

# URL de la página que contiene los inventarios de crudo
url = "https://es.investing.com/economic-calendar/eia-crude-oil-inventories-75"
driver.get(url)

# Esperar unos segundos para que la página cargue completamente
time.sleep(2)

# Clic en el botón "Mostrar más" varias veces para cargar más datos
while True:
    try:
        mostrar_mas = driver.find_element(By.LINK_TEXT, "Mostrar más")
        mostrar_mas.click()
        time.sleep(3)  # Esperar a que los nuevos datos se carguen
    except:
        break

# Una vez que todos los datos están cargados, obtenemos el contenido HTML
html = driver.page_source
soup = BeautifulSoup(html, "html.parser")

# Seleccionar las filas de la tabla que contienen los datos de inventario
rows = soup.select("#eventHistoryTable75 > tbody > tr")

# Lista para almacenar los datos extraídos
data = []

# Iterar sobre cada fila y extraer los datos
for row in rows:
    cols = row.find_all("td")
    if len(cols) > 0:
        # Extraer los datos de cada columna
        fecha = cols[0].get_text(strip=True)  # Fecha
        hora = cols[1].get_text(strip=True)   # Hora
        actual = cols[2].get_text(strip=True)  # Actual (columna 2)
        prevision = cols[3].get_text(strip=True)  # Previsión (columna 3)
        anterior = cols[4].get_text(strip=True)  # Anterior (columna 4)

        # Añadir a la lista de datos
        data.append({
            'Fecha de publicación': fecha,
            'Hora': hora,
            'Actual': actual,
            'Previsión': prevision,
            'Anterior': anterior
        })

# Convertir la lista a un DataFrame de pandas
df = pd.DataFrame(data)

# # Exportar como CSV
# df.to_csv('inventario-CL.csv', index=False)

# Cerrar el navegador
driver.quit()


In [23]:
df.to_csv('inventario-CL.csv', index=False)
df

,Fecha de publicación,Hora,Actual,Previsión,Anterior
0,11.09.2024,16:30,,,"-6,873M"
1,05.09.2024,17:00,"-6,873M","-0,600M","-0,846M"
2,28.08.2024,16:30,"-0,846M","-2,700M","-4,649M"
3,21.08.2024,16:30,"-4,649M","-2,000M","1,357M"
4,14.08.2024,16:30,"1,357M","-1,900M","-3,728M"
...,...,...,...,...,...
745,03.06.2010,17:00,"-1,900M",,"2,400M"
746,26.05.2010,16:30,"2,400M",,"0,200M"
747,19.05.2010,16:30,"0,200M",,"1,900M"
748,12.05.2010,16:30,"1,900M",,"2,800M"


In [ ]:
#Importamos el dataset scrapeado del inventario crudo semanal.
df_inventarios = pd.read_csv("inventario-CL.csv")

#Cambiamos nombre de la columna a Fecha
df_inventarios.rename(columns={
    "Fecha de publicación": "Fecha",
    "Actual": "Current_Inventory",
    "Previsión": "Current_Forecast",
    "Anterior": "Previous_Inventory"
}, inplace=True)

#Cambiamos a formato datime indicando que el el formato es europe con dia/mes/año
df_inventarios["Fecha"] = pd.to_datetime(df_inventarios["Fecha"],dayfirst = True)

In [ ]:
# Convertir las columnas a tipo string antes de aplicar la función ya que sino da error por ser "object"
df_inventarios[["Current_Inventory", "Current_Forecast", "Previous_Inventory"]] = df_inventarios[["Current_Inventory", "Current_Forecast", "Previous_Inventory"]].astype(str)

# Función para limpiar "M" y convertir "," en "." para que Python entienda decimales y luego multiplicar por 1 millón.
def limpiar_valores(x):
    # Si contiene "M", convertirlo a millones
    if "M" in x:
        return float(x.replace("M", "").replace(",", ".")) * 1_000_000
    # Si no contiene "M", convertirlo directamente
    return float(x.replace(",", "."))

# Aplicar la función a todas las columnas usando applymap() para las tres columnas
df_inventarios[["Current_Inventory", "Current_Forecast", "Previous_Inventory"]] = df_inventarios[["Current_Inventory", "Current_Forecast", "Previous_Inventory"]].applymap(limpiar_valores)

# Hacer el merge basado en la coincidencia de las fechas del dataset principal
df = pd.merge(df, df_inventarios, left_on="Date", right_on="Fecha", how="left")